# Momento de Retroalimentación: Módulo 2 Uso de framework o biblioteca de aprendizaje máquina para la implementación de una solución

Jorge Eduardo de León Reyna - A00829759

## Carga de librerias

In [51]:
import pandas as pd
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

## Descripcion del dataset utilizado

- **Dataset utilizado:** Heart Failure Prediction
- **Descripción y contexto del problema a resolver:** Las enfermedades cardiovasculares son la principal causa de muerte en todo el mundo, cobrando cerca de 17.9 millones de vidas al año y representando el 31% de todas las muertes. La mayoría de estas muertes son por ataques cardíacos y accidentes cerebrovasculares, y muchas ocurren prematuramente en personas menores de 70 años. La insuficiencia cardíaca también es común debido a estas enfermedades. Un conjunto de datos con 11 características puede ayudar a predecir posibles problemas cardíacos. Las personas con enfermedades cardíacas o alto riesgo cardiovascular, debido a factores como hipertensión o diabetes, necesitan detección temprana y manejo, donde un modelo de aprendizaje automático puede ser muy útil.
- **Atributos o campos:**
  - Age: edad del paciente
      - años
  - Sex: sexo del paciente
      - M: Male, F: Female
  - ChestPainType: tipo de dolor de pecho
      - TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic
  - RestingBP: presión arterial en reposo
      - mm Hg
  - Cholesterol: colesterol
      - mm/dl
  - FastingBS: azucar en la sangre en ayunas
      - 1: si FastingBS > 120 mg/dl, 0: otro
  - RestingECG: resultados de electrocardiograma en reposo
      - Normal: Normal, ST: Anomalía de la onda ST-T (inversiones de onda T y/o elevación o depresión del segmento ST > 0.05 mV), LVH: mostrando hipertrofia ventricular izquierda probable o definitiva según los criterios de Estes.
  - MaxHR: frecuencia cardíaca máxima alcanzada
      - Valor numero entre 60 y 202
  - ExerciseAngina: angina inducida por el ejercicio
      - Y: Yes, N: No
  - Oldpeak: Descenso antiguo = ST
      - Valor numerico
  - ST_Slope: La pendiente del segmento ST durante el ejercicio máximo
      - Up: upsloping, Flat: flat, Down: downsloping
  - HeartDisease: salida
      - 1: heart disease, 0: Normal
- **Enlace:** https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction?resource=download
- **Descripcion del problema:** En este dataset se busca hacer un modelo de clasificación, donde se tienen dos clases posibles para predecir: "heart disease(1)" y "normal(1)".








## Cargando Dataset

In [52]:
df = pd.read_csv("/content/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


#### Limpieza de datos inicial

In [53]:
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

## Matriz de correlación
Se busca encontrar las variables con mayor influencia sobre la variable objetivo ("charges") para utilizar las mismas en las pruebas del modelo.

In [54]:
correlation_matrix = df.corr()
print(correlation_matrix)

                   Age  RestingBP  Cholesterol  FastingBS     MaxHR   Oldpeak  \
Age           1.000000   0.254399    -0.095282   0.198039 -0.382045  0.258612   
RestingBP     0.254399   1.000000     0.100893   0.070193 -0.112135  0.164803   
Cholesterol  -0.095282   0.100893     1.000000  -0.260974  0.235792  0.050148   
FastingBS     0.198039   0.070193    -0.260974   1.000000 -0.131438  0.052698   
MaxHR        -0.382045  -0.112135     0.235792  -0.131438  1.000000 -0.160691   
Oldpeak       0.258612   0.164803     0.050148   0.052698 -0.160691  1.000000   
HeartDisease  0.282039   0.107589    -0.232741   0.267291 -0.400421  0.403951   

              HeartDisease  
Age               0.282039  
RestingBP         0.107589  
Cholesterol      -0.232741  
FastingBS         0.267291  
MaxHR            -0.400421  
Oldpeak           0.403951  
HeartDisease      1.000000  


<ipython-input-54-becb50483c66>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = df.corr()


Se encuentra que las variables con mayor correlacion a la variable objetivo ("charges") son: **"Oldpeak", "Age" y "FastingBS"** por lo que seran las seleccionadas para realizar pruebas con el modelo de regresion logistica.

## Preprocesamiento de datos y encoding

In [55]:
# Dividiendo dataset en features & lables
x = df[["Age", "Sex",	"ChestPainType", "RestingBP",	"Cholesterol",	"FastingBS",	"RestingECG", "MaxHR",	"ExerciseAngina",	"Oldpeak", "ST_Slope"]]
y = df["HeartDisease"]

# Definimos variables categoricos y numericos
categorical_features = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope" ]
numerical_features = ["Age","RestingBP","Cholesterol","FastingBS","MaxHR","Oldpeak"]


In [56]:

# Definimos pipeline para onehot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
    ],
    remainder='passthrough'
)

# Aplicamos one-hot encoding
x_encoded = preprocessor.fit_transform(x)

## Dividiendo dataset en set de entrenamiento y set de pruebas

In [60]:
# Dividimos datos de entrenamiento y en 3 subsets de pruebas
x_train, x_test1, y_train, y_test1 = train_test_split(x_encoded, y, test_size=0.3, random_state=42)
x_test2, x_test3, y_test2, y_test3 = train_test_split(x_test1, y_test1, test_size=0.5, random_state=42)



---



## Pipeline de entrenamiento y modificacion de hiperparametros

In [82]:
# Definimos  pipeline de entrenamiento y modificamos hiperparametros
model = Pipeline(steps=[
    ('classifier', LogisticRegression(
        penalty='l2', # Tipo de penalizacion
        C=1.0, # Parmetro de regularización
        solver='liblinear', # Solucionador
        max_iter=10000 # Numero de iteraciones
    ))
])

##Ejecucion de entrenamiento del modelo

In [83]:
model.fit(x_train, y_train)

Pipeline(steps=[('classifier',
                 LogisticRegression(max_iter=10000, solver='liblinear'))])

## Generacion de predicciones y evaluacion del modelo

#### Descripcion de metricas de evaluacion para sets de entrenamiento y prueba

Para estimar qué tan bueno es cada clasificador, debemos generar las matrices de confusión y las correspondientes métricas:

$accuracy = \frac{VP+VN}{VP+VN+FP+FN}$
- Accuracy (Exactitud): Calcula la proporción de predicciones correctas totales del modelo en relación con el número total de predicciones. Es útil cuando las clases están balanceadas.

$precision = \frac{VP}{VP+FP}$
- Precision (Precisión): Evalúa la proporción de instancias positivas que el modelo clasifica correctamente en relación con todas las instancias que predijo como positivas. Es especialmente importante cuando los falsos positivos son costosos o problemáticos.

$recall = \frac{VP}{VP+FN}$
- Recall (Recuerdo o Sensibilidad): Mide la proporción de positivos reales que el modelo identifica correctamente en relación con todos los positivos reales presentes en los datos.


$F1 = \frac{2\cdot precision \cdot recall}{precision+recall}$
- F1 Score: Es una métrica que combina precisión y recall para proporcionar una medida equilibrada del rendimiento del modelo, especialmente cuando hay un desequilibrio entre las clases.




#### Evaluacion de modelo con subset de prueba 1

In [84]:
# Predicciones con
y_pred1 = model.predict(x_test1)

In [85]:
# Matriz de confusión
confusion = confusion_matrix(y_test1, y_pred1)
print("Matriz de Confusión prueba 1:")
print(confusion)

Matriz de Confusión prueba 1:
[[100  12]
 [ 20 144]]


In [86]:
# Calculo de accuracy
accuracy = accuracy_score(y_test1, y_pred1)
print("Accuracy prueba 1:", accuracy)

Accuracy prueba 1: 0.8840579710144928


In [87]:
# Reporte de métricas (precision, recall, f1-score, support)
report = classification_report(y_test1, y_pred1)
print("Reporte de Clasificación Prueba 1:")
print(report)

Reporte de Clasificación Prueba 1:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86       112
           1       0.92      0.88      0.90       164

    accuracy                           0.88       276
   macro avg       0.88      0.89      0.88       276
weighted avg       0.89      0.88      0.88       276



#### Evaluacion de modelo con subset de prueba 2

In [88]:
# Predicciones con
y_pred2 = model.predict(x_test2)

In [89]:
# Matriz de confusión
confusion2 = confusion_matrix(y_test2, y_pred2)
print("Matriz de Confusión prueba 2:")
print(confusion2)

Matriz de Confusión prueba 2:
[[51  6]
 [11 70]]


In [90]:
# Calculo de accuracy
accuracy2 = accuracy_score(y_test2, y_pred2)
print("Accuracy prueba 2:", accuracy2)

Accuracy prueba 2: 0.8768115942028986


In [91]:
# Reporte de métricas (precision, recall, f1-score, support)
report2 = classification_report(y_test2, y_pred2)
print("Reporte de Clasificación Prueba 2:")
print(report2)

Reporte de Clasificación Prueba 2:
              precision    recall  f1-score   support

           0       0.82      0.89      0.86        57
           1       0.92      0.86      0.89        81

    accuracy                           0.88       138
   macro avg       0.87      0.88      0.87       138
weighted avg       0.88      0.88      0.88       138



#### Evaluacion de modelo con subset de prueba 3

In [92]:
# Predicciones con
y_pred3 = model.predict(x_test3)

In [93]:
# Matriz de confusión
confusion3 = confusion_matrix(y_test3, y_pred3)
print("Matriz de Confusión prueba 3:")
print(confusion3)

Matriz de Confusión prueba 3:
[[49  6]
 [ 9 74]]


In [94]:
# Calculo de accuracy
accuracy3 = accuracy_score(y_test3, y_pred3)
print("Accuracy prueba 3:", accuracy3)

Accuracy prueba 3: 0.8913043478260869


In [95]:
# Reporte de métricas (precision, recall, f1-score, support)
report3 = classification_report(y_test3, y_pred3)
print("Reporte de Clasificación Prueba 3:")
print(report3)

Reporte de Clasificación Prueba 3:
              precision    recall  f1-score   support

           0       0.84      0.89      0.87        55
           1       0.93      0.89      0.91        83

    accuracy                           0.89       138
   macro avg       0.88      0.89      0.89       138
weighted avg       0.89      0.89      0.89       138



#### Analisis de resultados de las metricas de evaluacion
 Despues de calcular los valores de la matriz de confusión para cada una de las implementaciones del modelo y el posterior calculo de las metricas de evaluacion en base a estas se encontró lo siguiente:

 1. Pese a haberse corrido en 3 subsets de datos distintos para probar el desempeño del modelo en contextos variados se encontró que su funcionamiento es adecuado al obtener el siguiente rango de resultados (en el reporte generado por sklearn se toma en cuenta la seccion de "wighted avg" dado el contexto del problema a resolver donde la precision del resultado en la clasificacion es muy importante):

  - **Accuracy:** valores entre 0.87 y 0.89, los cuales nos indican un buen funcionamiento correcto del modelo a primera instancia.
  - **Precision:** valores entre 0.88 y 0.89, lo que nos indica una buen nivel de prediccion en los valores positivos (1's) en comparacion a todas las muestras positivas (verdaderas y falsas).
  - **Recall:** valores entre 0.88 y 0.89, lo que nos indica una buen nivel de prediccion en los valores positivos verdaderos en comparacion al total de muestras positivas verdaderas (1's).
  - **F1-score:** valores entre 0.88 y 0.89, lo que nos indica un desempeño adecuado a nivel general a ser la medida armonica entre precision y recall.